In [2]:
# LINK GOOGLE DRIVE"

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Imports
import numpy as np
import pandas as pd
from tqdm import tqdm  
! pip install xmltodict
import xmltodict
! pip install sgp4
! pip install skyfield
from skyfield.api import load, wgs84
from skyfield.api import EarthSatellite
from datetime import datetime
import configparser
import json
import requests
import time
import urllib

     |████████████████████████████████| 258 kB 5.3 MB/s 
     |████████████████████████████████| 373 kB 5.3 MB/s 
     |████████████████████████████████| 40 kB 5.9 MB/s 
  Created wheel for skyfield: filename=skyfield-1.39-py3-none-any.whl size=417580 sha256=f61f2d938cf8e479ef976846dfddcdac5deaf105c78119443829d88aaba3cd49
  Stored in directory: /root/.cache/pip/wheels/56/d9/02/a7d09cef18b108aa82414257713cf2ad7c322f207cf55df113
  Created wheel for jplephem: filename=jplephem-2.16-py3-none-any.whl size=46793 sha256=76e6a6a7dd1ab1368b7eb7c31572df719830b38ccce4ef38f79cf72bb35bf594
  Stored in directory: /root/.cache/pip/wheels/c5/d6/01/722338266e80ec9e7a2faaf4e13fce9e6721fb0e2cd5308c7d
Successfully built skyfield jplephem


In [4]:
#opening the xml file in read mode
with open('/content/drive/MyDrive/CS685_project/data/doc.xml',"r") as xml_obj:
    #coverting the xml data to Python dictionary
    my_dict = xmltodict.parse(xml_obj.read())
    #closing the file
    xml_obj.close()

In [5]:
names= []
longitudes = []
latitudes = []
continent_list = my_dict['Document']['Folder']
for i in range(len(continent_list)-2):
    country_list = continent_list[i]['Placemark']
    for j in range(len(country_list)):
        name = country_list[j]['name']
        coordinates = str(country_list[j]['Point']['coordinates'])
        coordinates = coordinates.split(",")
        longitude = float(coordinates[0])
        latitude = float(coordinates[1])
        names.append(name)
        longitudes.append(longitude)
        latitudes.append(latitude)


country_list = continent_list[8]['Placemark']
for j in range(len(country_list)):
    name = country_list[j]['name']
    coordinates = str(country_list[j]['Point']['coordinates'])
    coordinates = coordinates.split(",")
    longitude = float(coordinates[0])
    latitude = float(coordinates[1])
    names.append(name)
    longitudes.append(longitude)
    latitudes.append(latitude)

name = my_dict['Document']['Folder'][7]['Placemark']['name']
coordinates = str(my_dict['Document']['Folder'][7]['Placemark']['Point']['coordinates'])
coordinates = coordinates.split(",")
longitude = float(coordinates[0])
latitude = float(coordinates[1])
names.append(name)
longitudes.append(longitude)
latitudes.append(latitude)


In [6]:
#MAKE CSV FROM ABOVE DATASHEET

dict = {
        'LAUNCH-SITE':names, 
        'LONGITUDE':longitudes, 
        'LATITUDE': latitudes,
       }
                  

dframe = pd.DataFrame(dict)   
dframe.to_csv('/content/drive/MyDrive/CS685_project/data/launsh_sites.csv')
dframe.head()

,LAUNCH-SITE,LONGITUDE,LATITUDE
0,Hammaguir,-2.835810,31.09951
1,Reggane,0.276910,26.71895
2,Kapani Tonneo,28.527660,-7.92587
3,Jabal Hamzah,30.605139,30.12575
4,San Marco platform,40.213400,-2.94080


In [7]:
df = pd.read_csv('/content/drive/MyDrive/CS685_project/data/filtered_elset.csv')
df.drop([df.columns[0]], axis=1 , inplace=True)
df.head()

,OBJECT_NAME,NORAD_ID,YEAR LAUNCH,EPOCH,INCLINATION(i),RIGHT ASCENSION ASCENDING NODE(w),ECCENTRICITY(e),ARGUMENT PERIGEE(omega),FIRST DERIVATIVE MEAN MOTION(n_dot),SECOND DERIVATIVE MEAN MOTION(n_ddot),MEAN ANOMALY(M),MEAN MOTION(n),DRAG TERM(B*),REVOLUTION(rev)
0,VANGUARD 1,5,58002B,21042.550355,34.2457,211.6733,0.184707,348.3317,-8.900000e-07,0.0,7.9494,10.848569,-10.5760,13249
1,VANGUARD 2,11,59001A,21042.875208,32.8652,236.4804,0.146771,163.1695,2.500000e-06,0.0,202.3396,11.857480,12.3720,34028
2,VANGUARD 3,20,59007A,21043.437696,33.3449,47.0561,0.166567,99.8465,4.900000e-07,0.0,279.3422,11.557979,2.6295,84140
3,EXPLORER 7,22,59009A,21042.889513,50.2840,58.4215,0.013875,93.7045,-1.600000e-07,0.0,267.9779,14.954023,2.2213,61910
4,TIROS 1,29,60002B,21042.696355,48.3785,259.5211,0.002395,177.0037,-1.170000e-06,0.0,183.1022,14.744034,1.3635,38438


In [8]:
# SPACE_TRACK API

class MyError(Exception):
    def __init___(self,args):
        Exception.__init__(self,"my exception was raised with arguments {0}".format(args))
        self.args = args


uriBase                = "https://www.space-track.org"
requestLogin           = "/ajaxauth/login"
requestCmdAction       = "/basicspacedata/query"

# Use configparser package to pull in the ini file (pip install configparser)
config = configparser.ConfigParser()
config.read("/content/drive/MyDrive/CS685_project/data/SLTrack.ini")
configUsr = config.get("configuration","username")
configPwd = config.get("configuration","password")
siteCred = {'identity': configUsr, 'password': configPwd}

def getTLESatellites(norad_id):

    with requests.Session() as session:
        # run the session in a with block to force session to close if we exit
        # need to log in first. note that we get a 200 to say the web site got the data, not that we are logged in
        resp = session.post(uriBase + requestLogin, data = siteCred)
        if resp.status_code != 200:
            raise MyError(resp, "POST fail on login")

        # this query picks up TLEs for all satellites from the catalog. Note - a 401 failure shows you have bad credentials
        # Enter the fields for which query must be built

        query_link = ("/class/tle/NORAD_CAT_ID/" + str(norad_id) + "/orderby/EPOCH asc/limit/1/format/3le/emptyresult/show")

        resp = session.get(uriBase + requestCmdAction + query_link)
        
        if resp.status_code != 200:
            time.sleep(5)
            return getTLESatellites(norad_id)

        # resp.text contains the required Tles
        data = resp.text
        session.close()
    return data

In [9]:
df2 = pd.read_csv('/content/drive/MyDrive/CS685_project/data/launch_dates.csv')
df2.drop([df2.columns[0]], axis=1 , inplace=True)
df2.head()

,Name,Launch date (YYYY-MM-DD),Status,Action
0,SPUTNIK 1,1957-10-04,DECAYED,NaN
1,SPUTNIK 2,1957-11-03,DECAYED,NaN
2,EXPLORER 1,1958-02-01,DECAYED,NaN
3,VANGUARD 1,1958-03-17,IN ORBIT,TRACK IT
4,EXPLORER 3,1958-03-26,DECAYED,NaN


In [10]:
def find_closest_subpoint(longitude, latitude):
    long_threshold = 2.5 # deg
    lat_threshold = 2.5 # deg
    closest_point = None
    min_dev = float("inf")
    for i in range(dframe.shape[0]):
        launch_site_long = dframe.iloc[i][1]
        launch_site_lat = dframe.iloc[i][2]

        if abs(launch_site_long - longitude)<=long_threshold and abs(launch_site_lat - latitude)<=lat_threshold:
            if abs(launch_site_long - longitude)<=long_threshold  + abs(launch_site_lat - latitude)<=lat_threshold < min_dev:
                min_dev = abs(launch_site_long - longitude)<=long_threshold  + abs(launch_site_lat - latitude)<=lat_threshold
                closest_point = df.iloc[i][0]

    return closest_point

In [12]:
epoch_list = []
tle_line1_list =[]
tle_line2_list = []
tle_line3_list = []
launch_sites= []

for i in tqdm(range(df.shape[0])):
    time.sleep(2)
    norad_id = df.iloc[i]['NORAD_ID']
    starting_tles = getTLESatellites(norad_id)

    lines = starting_tles.splitlines()

    tle_line1 = lines[0]
    tle_line2 = lines[1]
    tle_line3 = lines[2]

    tle_line2_ = tle_line2.split()
    epoch = float(tle_line2_[4])
    epoch_list.append(epoch)
    tle_line1_list.append(tle_line1)
    tle_line2_list.append(tle_line2)
    tle_line3_list.append(tle_line3)

 18%|█▊        | 1233/6952 [59:59<2:46:27,  1.75s/it]

In [ ]:
for i in tqdm(range(df.shape[0])):
    epoch = epoch_list[i]
    tle_line1 = tle_line1_list[i]
    tle_line2 = tle_line2_list[i]
    tle_line3 = tle_line3_list[i]
    ts = load.timescale()
    satellite_skyfield = EarthSatellite(tle_line2, tle_line3, tle_line1, ts)
    geocentric = satellite_skyfield.at(satellite_skyfield.epoch)
    subpoint = wgs84.subpoint(geocentric)
 
    dte = datetime.strptime('2000-01-01 01:01:01',"%Y-%m-%d %H:%M:%S")
    epoch_ = dte.replace(minute=satellite_skyfield.epoch.utc.minute, hour=satellite_skyfield.epoch.utc.hour,
						 second=int(satellite_skyfield.epoch.utc.second), year=satellite_skyfield.epoch.utc.year,
						  month=satellite_skyfield.epoch.utc.month, day=satellite_skyfield.epoch.utc.day)
 
    long = subpoint.longitude.degrees
    lat = subpoint.latitude.degrees

    launch_site = find_closest_subpoint(long,lat)
    launch_sites.append(launch_site)

In [ ]:
launch_sites= []

for i in tqdm(range(df2.shape[0])):
    satname = df2.iloc[0][i]
    launch_date = df2.iloc[1][i]
    date = datetime.fromisoformat(date_string)

    ts = load.timescale()
    t = ts.utc(date.year, date.month, date.day, 12,12,12)
    satellite_skyfield = EarthSatellite(tle_line2, tle_line3, tle_line1, ts)
    geocentric = satellite_skyfield.at(t)
    subpoint = wgs84.subpoint(geocentric)
 
    long = subpoint.longitude.degrees
    lat = subpoint.latitude.degrees

    launch_site = find_closest_subpoint(long,lat)
    launch_sites.append(launch_site) 

In [ ]:
df['MOST PROBABLE LAUNCH SITE'] = launch_sites

In [ ]:
ts = load.timescale()
    satellite = EarthSatellite(tle_line1, tle_line2, object_name_, ts)
    year = 2014
    month = 1
    date = 23
    hour = 12
    min = 12
    sec = 12
    t = ts.utc(year, month, date, hour, min, sec)
    geocentric = satellite.at(t)
    subpoint = wgs84.subpoint(geocentric)
    print(subpoint)